# Marching Squares
Takes a scalar field such as the Signed Distance Function (SDF) and generates contours consisting of line segment. A similar algorithm exists in 3D where a scalar field is turned into surfaces consisting of triangles.

In [ ]:
import numpy as np

In [ ]:
import seaborn as sns
sns.set_theme()
sns.set(style='darkgrid', context='talk', palette='Pastel1')
sns.set(style='dark')  # get rid of gridlines

In [ ]:
# configure matlab for notebooks
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [12, 5]

In [ ]:
# load points cloud from the field interpolation cookie talk
sdf = np.load('sdf-512-16.00.npy')
sdf.shape

In [ ]:
from matplotlib import colormaps
cmap = colormaps['Spectral']
#cmap = sns.diverging_palette(105, 290, 90, 70, center="dark", as_cmap=True)
plt.imshow(sdf, cmap=cmap, vmin=-7, vmax=7); plt.colorbar()

## Algorithm
* Each 2x2 block of the SDF is iterated
* Each block is thresholded against the level (e.g. 0) as either below or not below.
* There are 16 (2^4) different configurations each yielding a specific set of lines (or no line) cutting the square
* The exact vertex positions of the lines are computed by lineary interpolating the positions

In [ ]:
from mpl_toolkits.axes_grid1.inset_locator import mark_inset, zoomed_inset_axes

h, w = sdf.shape
x, y = 8, 8  # top-left cordinate of crop-out
zw, zh = 2, 2  # size of out-crop
plt.imshow(sdf, origin='upper'); plt.colorbar()
axins = plt.gca().inset_axes(
    [0.54, 0.5, 0.47, 0.47],
    xlim=(x, x+zw-1), ylim=(y, y+zh-1),
    xticklabels=[], yticklabels=[],
)
axins.imshow(sdf, origin='upper')
mark_inset(plt.gca(), axins, loc1=2, loc2=3, edgecolor="0.6")
plt.show()

In [ ]:
# Vertex positions within a single 2x2 block
#
# 0 -- 1
# |    |
# 2 -- 3
POINTS = np.array([
    (0, 0),
    (1, 0),
    (0, 1),
    (1, 1),
])

In [ ]:
values = np.array([
    sdf[y, x],
    sdf[y, x + 1],
    sdf[y + 1, x],
    sdf[y + 1, x + 1],
])
print(x, y)
print(values)
print(values < 0)

In [ ]:
bits = values < 0

plt.gca().set_box_aspect(1)
plt.gca().invert_yaxis()
plt.scatter(POINTS[bits, 0], POINTS[bits, 1], facecolors='black', edgecolors='black')
plt.scatter(POINTS[~bits, 0], POINTS[~bits, 1], facecolors='none', edgecolors='black')


In [ ]:
# A list of the sixteen variants.
# Each entry contains a sequence of pairs of edges. For example entry #1 contains one entry with the edge pair
# (0, 2) and (0, 1). This will create a line between edges left edge (0, 2) and the top edge (0, 1). Note that 
# two variants contains no images (when all four values are either below or above the level). Also two variants will
# yield two lines
LOOKUP = [
    (),
    (((0, 2), (0, 1)),),
    (((0, 1), (1, 3)),),
    (((0, 2), (1, 3)),),
    (((0, 2), (2, 3)),),
    (((0, 1), (2, 3)),),
    (((0, 2), (2, 3)), ((0, 1), (1, 3))),
    (((2, 3), (1, 3)),),
    (((1, 3), (2, 3)),),
    (((0, 2), (0, 1)), ((2, 3), (1, 3))),
    (((2, 3), (0, 1)),),
    (((2, 3), (0, 2)),),
    (((1, 3), (0, 2)),),
    (((1, 3), (0, 1)),),
    (((0, 1), (0, 2)),),
    (),    
]

In [ ]:
def average_edge(points: np.ndarray, edge: Edge) -> np.ndarray:
    i0, i1 = edge
    return (points[i0] + points[i1]) * 0.5

In [ ]:
def to_bits(number: int, n: int) -> np.ndarray:
    return np.array([bool(number & (1 << i)) for i in range(n)], dtype=bool)

fig, axes = plt.subplots(4, 4)
plt.subplots_adjust(wspace=0.5, hspace=0.8)
for i in range(16):
    row, column = i // 4, i % 4  # layout the variants in a 4x4 grid
    plot = axes[row, column]
    plot.set_title(f'Case {i}')
    margin = 0.1
    plot.axis('off'); plot.axis('equal'); plot.axis([0 - margin, 1 + margin, 1 + margin, 0 - margin])

    bits = to_bits(i, n=4)  # find bits that contain points with value BELOW level
    plot.scatter(POINTS[bits, 0], POINTS[bits, 1], facecolors='black', edgecolors='black')
    plot.scatter(POINTS[~bits, 0], POINTS[~bits, 1], facecolors='none', edgecolors='black')
    for e0, e1 in LOOKUP[i]:
        a, b = average_edge(POINTS, e0), average_edge(POINTS, e1)
        plot.plot([a[0], b[0]], [a[1], b[1]], color='purple')

In [ ]:
# Quick bit recap
tmp = [True, False, True, True]
for i, bit in enumerate(tmp):
    print(i, 1 << i, int(bit), bit)

In [ ]:
from typing import Iterable, Tuple


def iter_blocks(array: np.ndarray, block_shape: Tuple[int, int]) -> Iterable:
    """Slides a window over the array and yields all blocks and their top-left cordinates"""
    bh, bw = block_shape
    for y in range(h - bh):
        for x in range(w - bw):
            yield np.array([x, y]), array[y:y+bh, x:x+bw]


def marching_squares_nolerp(sdf: np.ndarray, level: float = .0) -> Tuple[np.ndarray, np.ndarray]:
    h, w = sdf.shape
    vertices = []
    lines = []
    for p, block in iter_blocks(sdf, (2, 2)):
        # threshold SDF values
        values = block < level
        # compute variant number
        index = sum(int(bit) * 1 << i for i, bit in enumerate(values.ravel()))
        #p = np.array([x, y])
        for e0, e1 in LOOKUP[index]:
            a, b = average_edge(POINTS, e0), average_edge(POINTS, e1)
            lines.append((len(vertices), len(vertices) + 1))
            vertices.extend([p + a, p + b])

    return np.vstack(vertices), lines

vertices, lines = marching_squares_nolerp(sdf)

In [ ]:
from PIL import Image
from PIL import ImageDraw

def draw_lines(vertices: np.ndarray, lines: List[Tuple[int, int]], linewidth: int = 2) -> None:
    im = Image.new('RGB', (1024, 512), (0xff, 0xff, 0xff))
    draw = ImageDraw.Draw(im)
    color = (0x00, 0x00, 0x00)
    scale = 16
    for a, b in lines:
        draw.line((tuple(scale * vertices[a]), tuple(scale * vertices[b])), fill=color, width=linewidth)
    return im

draw_lines(vertices, lines)

## Linear interpolation (lerp)

Given two values, $a$ and $b$ and a parameter $t \in [0, 1]$ generate a new value.

$$
f(a, b, t) = (1 - t) * a + t * b
$$

## Interpolating the SDF values
$$t = (l - v_0) / (v_1 - v_0)$$

In [ ]:
x = np.array([-2, 0, 3])
y = np.zeros_like(x)
labels = ['v0', 'level', 'v1']

fig, ax = plt.subplots()
ax.vlines(x, -3, 5)
ax.set_yticks([])
for xx, yy, label in zip(x, y, labels):
    ax.annotate(label, (xx, yy))


In [ ]:
def interpolate_edge(points: np.ndarray, values: np.ndarray, edge: Edge, level: float) -> np.ndarray:
    i0, i1 = edge
    v0, v1 = values[i0], values[i1]
    # make sure v0 is smaller that v1
    if v0 > v1:
        i0, i1 = i1, i0
        v0, v1 = v1, v0

    # compute parameter
    t = (level - v0) / (v1 - v0)
    # lerp
    return (1 - t) * points[i0] + t * points[i1]


def marching_squares(sdf: np.ndarray, level: float = .0) -> Tuple[np.ndarray, np.ndarray]:
    h, w = sdf.shape
    vertices = []
    lines = []
    for p, block in iter_blocks(sdf, (2, 2)):
        values = block.ravel()
        index = sum(int(bit) * 1 << i for i, bit in enumerate(values < level))
        for e0, e1 in LOOKUP[index]:
            a, b = interpolate_edge(POINTS, values, e0, level), interpolate_edge(POINTS, values, e1, level)
            lines.append((len(vertices), len(vertices) + 1))
            vertices.extend([p + a, p + b])

    return np.vstack(vertices), lines


vertices, lines = marching_squares(sdf)
draw_lines(vertices, lines)

## From squares to cubes
The corresponding 3D algorithm
* Iterates _blocks_ of 2x2x2 grid cells (cubes) rather than 2x2 grid cells (squares).
* Generates triangles instead of lines

## Quiz time!

Linear interpolation...
1. Predicts a new value based on two previous values (a, b)
2. Generates a new values between two given values (a, b)
3. Is a way to scale an image (image a to image b)

Marching squares requires...
1. Advanced math
2. GPU processing
3. Clever book-keeping



## Questions?